In [2]:
import pandas as pd
import requests
import json
import os
from bs4 import BeautifulSoup
import re
from queue import Queue
from threading import Thread
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
json1_file = open(os.getcwd() + '/Credentials.json')
json1_str = json1_file.read()
credential = json.loads(json1_str)
Genius_Token = credential['Genius_API']['Access_Token']

In [177]:
def request_song_info(song_id,Genius_Token):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    song_url = base_url + '/songs/' + str(song_id)
    response = requests.get(song_url, headers = headers)

    return response

In [5]:
def get_song_id(ix,song_title,artist_title,Genius_Token):
    
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    search_url = base_url + '/search'
    data = {'q': str(song_title) + ' ' + str(artist_title)}
    response = requests.get(search_url, data = data, headers=headers)

    if response.ok == True:
        song_info = response.json()
        if len(song_info['response']['hits']) != 0:
            song_id = song_info['response']['hits'][0]['result']['id']

        else:
            song_id = 'Not Found'
        return (ix,song_id)
    else:
        return "Response Error!"
    

In [6]:
def is_legal_string(test_string):
    legal_characters = r'\w+$'
    r = re.compile(legal_characters)
    
    if re.match(r, test_string):
        return 1
    else:
        return 0

In [7]:
def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics

In [178]:
def get_lyrics(song_id):
    response = request_song_info(song_id,Genius_Token)
    if response.ok == True:
        info = response.json()
        if len(info['response']['song']['url']) != 0:
            song_url = info['response']['song']['url']
            lyrics = scrap_song_url(song_url)
        else:
            lyrics = 'Not Found'
    else:
        return "Response Error!"
    
    
    return lyrics

In [9]:
def get_release_date(song_id):
    song_info = request_song_info(song_id,Genius_Token)
    song_info = song_info.json()
    
    return song_info['response']['song']['release_date']

In [10]:
tracks = pd.read_csv(r'Clean Data/track_with_artists.csv')

In [114]:
result = list()
def worker():
    while True:
        item = q.get()
        result.append(get_song_id(item[0],item[1],item[2],Genius_Token))
        q.task_done()

q = Queue()
for i in range(8):
    t = Thread(target=worker)
    t.daemon = True
    t.start()

for item in tracks.iterrows():
    q.put((item[0],item[1]['TITLE'],item[1]['ARTIST']))

q.join() 

In [139]:
result = pd.DataFrame({'ix':[i[0] for i in result],'song_id':[i[1] for i in result]})
tracks_with_song_ids = pd.merge(result,tracks,left_on = 'ix',right_index = True, how = 'inner').drop('ix',axis = 1)

In [141]:
tracks_with_song_ids.to_csv(r'tracks_with_song_ids.csv',index = False)

In [164]:
song_id_no_duplicates = tracks_with_song_ids[tracks_with_song_ids['song_id'].duplicated() == False]
song_id_no_duplicates.to_csv('Clean Data/song_id_no_duplicates.csv',index = False)

In [165]:
song_id_duplicates = tracks_with_song_ids[tracks_with_song_ids['song_id'].duplicated() == True]
song_id_duplicates.to_csv('Clean Data/song_id_duplicates.csv',index = False)

In [166]:
len(song_id_no_duplicates) + len(song_id_duplicates) == len(tracks_with_song_ids)

True

In [168]:
print("unique song_ids found:{}".format(len(song_id_no_duplicates)))

unique song_ids found:286110


In [170]:
song_id_no_duplicates.head()

,song_id,ISRC,SPOTIFY_ID,TITLE,ARTIST
0,3035330,USRC11702155,1j4kHkkpqZRBwE0A4CN4Yv,Dusk Till Dawn - Radio Edit,ZAYN
1,3646568,USUM71805170,7dt6x5M1jzdTEt8oCbisTK,Better Now,Post Malone
2,2942139,USUM71710087,7wGoVu4Dady5GV0Sv4UIsx,rockstar,Post Malone
3,2992529,USSM11706905,0ofbQMrRDsUaVKq2mGLEAb,Havana,Camila Cabello
4,3188009,USRC11701901,7vGuf3Y35N4wmASOKLUVVU,Silence,Marshmello


In [200]:
result = list()
def worker():
    while True:
        item = q.get()
        result.append([item[0],get_lyrics(item[0])])
        q.task_done()

q = Queue()
for i in range(8):
    t = Thread(target=worker)
    t.daemon = True
    t.start()

for item in song_id_no_duplicates.iterrows():
    q.put(item[1]['song_id'])

q.join() 

Exception in thread Thread-59:
Traceback (most recent call last):
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 444, in wrap_socket
    cnx.do_handshake()
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1426, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1166, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (54, 'ECONNRESET')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/Users/Stella/miniconda3/lib/python3.6/site-pack

Exception in thread Thread-56:
Traceback (most recent call last):
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 444, in wrap_socket
    cnx.do_handshake()
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1426, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1166, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (54, 'ECONNRESET')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/Users/Stella/miniconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/Users/Stella/miniconda3/lib/python3.6/site-pack

KeyboardInterrupt: 